In [29]:
import io
import os
import re
import shutil
import numpy as np
import pandas as pd
from joblib import dump, load
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor

In [54]:
# 1.g2_grid_search. finish the fpParas.dat all from g1-opt.

num_zeta = [1, 10, 50, 100, 200, 400, 600, 800, 1000]
num_thetas = [0.55, 0.6]
ori_path = '/Users/renzhe/Desktop/g2/fpParas_all/fpParas_from_g1.dat'

for zeta in num_zeta:
    for ts in num_thetas:
        new_path = '/Users/renzhe/Desktop/g2/fpParas_all/{}-{}.dat'.format(zeta,ts)
        with open(new_path, 'w') as file_object:
            # 1.write G1 and change G2_number.
            f_old = open(ori_path,'r')
            code = []
            for line in f_old:
                code.append(line)  
            t_i = np.arange(0,np.pi,ts)
            code[6] = '9 {}\n'.format(len(t_i))
            for line_ in code:    
                file_object.write(line_)
            # 2. write G2.
            num_ = 0
            for t_ii in t_i:
                if num_ == len(t_i) - 1:
                    file_object.write('G2      Ge       Ge       Ge     0.004     {}     1.00     {}   5.7'.
                                  format(zeta,np.round(t_ii,3)))
                else:
                    file_object.write('G2      Ge       Ge       Ge     0.004     {}     1.00     {}   5.7\n'.
                                  format(zeta,np.round(t_ii,3)))
                num_ += 1
        file_object.close()
print('Finish') 

Finish


In [39]:
# 2.pyamff training. 72个pyamff作业
num_zeta = [10, 50, 100, 200, 400, 600, 800, 1000] #8个
num_thetas = [0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5] # 9个


# 2.1 建立文件夹
# print('cd /work/mse-minzw/lrz/g1/g2/zata_thetas')
# for zeta in num_zeta:
#     for ts in num_thetas:
#         print('mkdir {}-{}'.format(zeta, ts))


# 2.2 移动文件
# for zeta in num_zeta:
#     for ts in num_thetas:
#         print('cd /work/mse-minzw/lrz/g1/g2/zata_thetas/{}-{}'.format(zeta, ts))
#         print('cp /work/mse-minzw/lrz/g1/g2/zata_thetas/data_all/fpParas_all/{}-{}.dat .'.
#               format(zeta, ts))
#         print('cp /work/mse-minzw/lrz/g1/data_all/g1_1/g1_1_dRs/drs100/0.475/recal3/config.ini .')
#         print('cp /work/mse-minzw/lrz/g1/data_all/g1_1/g1_1_dRs/drs100/0.475/recal3/run-pyamff.sh .')
#         print('cp /work/mse-minzw/lrz/g1/data_all/g1_1/g1_1_dRs/drs100/0.475/recal3/pred.py .')
#         print('cp /work/mse-minzw/lrz/g1/data_all/g1_1/g1_1_dRs/drs100/0.475/recal3/run-pred.sh .')
#         print('mv {}-{}.dat fpParas.dat'.format(zeta, ts))
#         print(' ')


# 2.3 pyamff-training-work
# for zeta in num_zeta:
#     for ts in num_thetas:
#         print('cd /work/mse-minzw/lrz/g1/g2/zata_thetas/{}-{}'.format(zeta, ts))
#         print('bsub < run-pyamff.sh')
#         print(' ')
 
    
# 2.4 prediction-work
# for zeta in num_zeta:
#     for ts in num_thetas:
#         print('cd /work/mse-minzw/lrz/g1/g2/zata_thetas/{}-{}'.format(zeta, ts))
#         print('bsub < run-pred.sh')
#         print(' ')


# 2.5 批量整理data，准备优化不合理的点
import io
import os
import re
import shutil
import numpy as np
import pandas as pd

def read_rmse(path='/work/mse-minzw/lrz/g1/g2/zata_thetas'):
    index_ = []
    num_zeta = [10, 50, 100, 200, 400, 600, 800, 1000] # 8个
    num_thetas = [0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5] # 9个
    for zeta in num_zeta:
        for ts in num_thetas:
            index_.append('{}-{}'.format(zeta, ts))
    df_ef_opt = pd.DataFrame(index=index_, columns=['te','tf','pe','pf',])
    for zeta in num_zeta:
        for ts in num_thetas:
            path_pl = path + '/{}-{}'.format(zeta, ts) + '/pyamff.log'
            file = open(path_pl)
            out = file.readlines()
            rmse = out[-1]
            num_list = rmse.split()
            EnergyRMSE = float(num_list[-2])
            ForceRMSE = float(num_list[-1])
            df_ef_opt.loc['{}-{}'.format(zeta, ts), 'te'] = EnergyRMSE
            df_ef_opt.loc['{}-{}'.format(zeta, ts), 'tf'] = ForceRMSE
            print('Training', zeta, '-', ts)
    for zeta in num_zeta:
        for ts in num_thetas:            
            path_prl = path + '/{}-{}'.format(zeta, ts) + '/pred.log'
            pfile = open(path_prl)
            out = pfile.readlines()
            rmse = out[-65]
            pattern = r'[\d.]+'
            number = re.findall(pattern, rmse)
            EnergyRMSE_str = number[0]
            ForceRMSE_str = number[1]
            EnergyRMSE = float(EnergyRMSE_str)
            ForceRMSE = float(ForceRMSE_str)
            df_ef_opt.loc['{}-{}'.format(zeta, ts), 'pe'] = EnergyRMSE
            df_ef_opt.loc['{}-{}'.format(zeta, ts), 'pf'] = ForceRMSE 
            print('Prediction', zeta, '-', ts)        
    with pd.ExcelWriter('./g2_map_opt.xlsx') as writer1:
        df_ef_opt.to_excel(writer1, sheet_name='g2_opt')
        
read_rmse()        


In [ ]:
# 3. g2_map_opt

# firstly: def function to statistic RMSE in recal file.

import io
import os
import re
import shutil
import numpy as np
import pandas as pd

def recal_RMSE(R_num=1):
    index_ = []
    recal_num = R_num
    path0 = '/work/mse-minzw/lrz/g1/g2/zata_thetas/'
    num_zeta = [10, 50, 100, 200, 400, 600, 800, 1000] #8个
    num_thetas = [0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5] # 9个
    for zeta in num_zeta:
        for ts in num_thetas:
            path_recal = 'recal{}'.format(recal_num)
            path_zata_ts = '/work/mse-minzw/lrz/g1/g2/zata_thetas/{}-{}'.format(zeta, ts)
            if os.path.exists(os.path.join(path_zata_ts, path_recal)):
                index_.append('{}-{}'.format(zeta, ts))                    
    df_ef_opt = pd.DataFrame(index=index_, columns=['te','tf','pe','pf'])            
    for p_recal in index_:
        path_prl = path0 + p_recal + '/recal{}'.format(recal_num) + '/pred.log'
        path_lrz = path0 + p_recal + '/recal{}'.format(recal_num) + '/pyamff.log'
        file = open(path_lrz)
        out = file.readlines()
        rmse = out[-1]
        num_list = rmse.split()
        ForceRMSE = float(num_list[-1])
        EnergyRMSE = float(num_list[-2])
        df_ef_opt.loc['{}'.format(p_recal), 'tf'] = ForceRMSE
        df_ef_opt.loc['{}'.format(p_recal), 'te'] = EnergyRMSE
        print('Finish Training ', '{}'.format(p_recal)) 
        
        pfile = open(path_prl)
        outp = pfile.readlines()
        rmsep = outp[-36]
        pattern = r'[\d.]+'
        number = re.findall(pattern, rmsep)
        ForceRMSE_str = number[1]
        EnergyRMSE_str = number[0]
        ForceRMSEp = float(ForceRMSE_str)
        EnergyRMSEp = float(EnergyRMSE_str)
        df_ef_opt.loc['{}'.format(p_recal), 'pf'] = ForceRMSEp
        df_ef_opt.loc['{}'.format(p_recal), 'pe'] = EnergyRMSEp
        print('Finish Prediction ', '{}'.format(p_recal))  
        
    with pd.ExcelWriter('./g2_map_opt_recal{}.xlsx'.format(recal_num)) as writer1:
        df_ef_opt.to_excel(writer1, sheet_name='g2_opt_recal{}'.format(recal_num))


recal_RMSE(R_num=1)


In [66]:
# # 3.1 g2_map_opt_recal1
# import pandas as pd
# data_p_f = pd.read_excel('/Users/renzhe/Desktop/g2/g2_map_opt.xlsx',sheet_name='g2_opt')
# for i in range(1,len(data_p_f)-1):
#     _a = data_p_f.iloc[i-1,-1]
#     a  = data_p_f.iloc[i+0,-1]
#     a_ = data_p_f.iloc[i+1,-1]
#     if (_a<a) and (a_<a):
#         pathi = data_p_f.iloc[i,0]
#         print('cd /work/mse-minzw/lrz/g1/g2/zata_thetas/{}'.format(pathi))
#         print('mkdir recal1')
#         print('cd recal1')
        
#         print('cp ../pred.py .')
#         print('cp ../config.ini .')
#         print('cp ../fpParas.dat .')
#         print('cp ../run-pred.sh .')
#         print('cp ../run-pyamff.sh .')
#         print('bsub < run-pyamff.sh')
#         print('  ')

# # 3.1 g2_map_opt_recal1
# import pandas as pd
# data_p_f = pd.read_excel('/Users/renzhe/Desktop/g2/g2_map_opt.xlsx',sheet_name='g2_opt')
# for i in range(1,len(data_p_f)-1):
#     _a = data_p_f.iloc[i-1,-1]
#     a  = data_p_f.iloc[i+0,-1]
#     a_ = data_p_f.iloc[i+1,-1]
#     if (_a<a) and (a_<a):
#         pathi = data_p_f.iloc[i,0]
#         print('cd /work/mse-minzw/lrz/g1/g2/zata_thetas/{}/recal1'.format(pathi))
#         print('bsub < run-pred.sh')
#         print('  ')

In [67]:
# # 3.2 g2_map_opt_recal2
# recalnum = 2
# num_zeta = [10, 50, 100] #3个
# num_thetas = [0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5] # 9个

# for zeta in num_zeta:
#     for ts in num_thetas:
#         print('cd /work/mse-minzw/lrz/g1/g2/zata_thetas/{}-{}'.format(zeta, ts))
#         print('mkdir recal{}'.format(recalnum))
#         print('cd recal{}'.format(recalnum))
#         print('cp ../pred.py .')
#         print('cp ../config.ini .')
#         print('cp ../fpParas.dat .')
#         print('cp ../run-pred.sh .')
#         print('cp ../run-pyamff.sh .')
#         print('bsub < run-pyamff.sh')
#         print('  ')

# # 3.2 g2_map_opt_recal2
# recalnum = 2
# num_zeta = [10, 50, 100] #3个
# num_thetas = [0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5] # 9个

# for zeta in num_zeta:
#     for ts in num_thetas:
#         print('cd /work/mse-minzw/lrz/g1/g2/zata_thetas/{}-{}/recal2'.format(zeta, ts))
#         print('bsub < run-pred.sh')
#         print('  ')

In [12]:
# # 3.3 g2_map_opt_recal3
# import pandas as pd
# data_p_f = pd.read_excel('/Users/renzhe/Desktop/g2/g2_map_opt.xlsx',sheet_name='g2_opt')
# for i in range(1,len(data_p_f)-1):
#     _a = data_p_f.iloc[i-1,-1]
#     a  = data_p_f.iloc[i+0,-1]
#     a_ = data_p_f.iloc[i+1,-1]
#     if (_a<a) and (a_<a):
#         pathi = data_p_f.iloc[i,0]
#         print('cd /work/mse-minzw/lrz/g1/g2/zata_thetas/{}'.format(pathi))
#         print('mkdir recal3')
#         print('cd recal3')
        
#         print('cp ../pred.py .')
#         print('cp ../config.ini .')
#         print('cp ../fpParas.dat .')
#         print('cp ../run-pred.sh .')
#         print('cp ../run-pyamff.sh .')
#         print('bsub < run-pyamff.sh')
#         print('  ')

# # 3.1 g2_map_opt_recal3
# import pandas as pd
# data_p_f = pd.read_excel('/Users/renzhe/Desktop/g2/g2_map_opt.xlsx',sheet_name='g2_opt')
# for i in range(1,len(data_p_f)-1):
#     _a = data_p_f.iloc[i-1,-1]
#     a  = data_p_f.iloc[i+0,-1]
#     a_ = data_p_f.iloc[i+1,-1]
#     if (_a<a) and (a_<a):
#         pathi = data_p_f.iloc[i,0]
#         print('cd /work/mse-minzw/lrz/g1/g2/zata_thetas/{}/recal1'.format(pathi))
#         print('bsub < run-pred.sh')
#         print('  ')


In [13]:
# # 3.3 g2_map_opt_recal5
# import pandas as pd
# data_p_f = pd.read_excel('/Users/renzhe/Desktop/g2/g2_map_opt.xlsx',sheet_name='g2_opt')
# for i in range(1,len(data_p_f)-1):
#     a  = data_p_f.iloc[i+0,-1]
#     if a > 0.147:
#         pathi = data_p_f.iloc[i,0]
#         print('cd /work/mse-minzw/lrz/g1/g2/zata_thetas/{}/recal6'.format(pathi))
#         print('bsub < run-pred.sh')
#         print('  ')

In [11]:
# # 3.2 g2_map_opt_recal6
# recalnum = 6
# num_zeta = [50, 100] #2个
# num_thetas = [0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5] # 9个

# for zeta in num_zeta:
#     for ts in num_thetas:
#         print('cd /work/mse-minzw/lrz/g1/g2/zata_thetas/{}-{}'.format(zeta, ts))
#         print('mkdir recal{}'.format(recalnum))
#         print('cd recal{}'.format(recalnum))
#         print('cp ../pred.py .')
#         print('cp ../config.ini .')
#         print('cp ../fpParas.dat .')
#         print('cp ../run-pred.sh .')
#         print('cp ../run-pyamff.sh .')
#         print('bsub < run-pyamff.sh')
#         print('  ')


# recalnum = 6
# num_zeta = [50, 100] # 2个
# num_thetas = [0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5] # 9个

# for zeta in num_zeta:
#     for ts in num_thetas:
#         print('cd /work/mse-minzw/lrz/g1/g2/zata_thetas/{}-{}/recal6'.format(zeta, ts))
#         print('bsub < run-pred.sh')
#         print('  ')

In [10]:
# # 3.2 g2_map_opt_recal7
# recalnum = 7
# num_zeta = [50, 100] # 2个
# num_thetas = [0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5] # 9个

# for zeta in num_zeta:
#     for ts in num_thetas:
#         print('cd /work/mse-minzw/lrz/g1/g2/zata_thetas/{}-{}'.format(zeta, ts))
#         print('mkdir recal{}'.format(recalnum))
#         print('cd recal{}'.format(recalnum))
#         print('cp ../pred.py .')
#         print('cp ../config.ini .')
#         print('cp ../fpParas.dat .')
#         print('cp ../run-pred.sh .')
#         print('cp ../run-pyamff.sh .')
#         print('bsub < run-pyamff.sh')
#         print('  ')


# recalnum = 7
# num_zeta = [50, 100] # 2个
# num_thetas = [0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5] # 9个

# for zeta in num_zeta:
#     for ts in num_thetas:
#         print('cd /work/mse-minzw/lrz/g1/g2/zata_thetas/{}-{}/recal7'.format(zeta, ts))
#         print('bsub < run-pred.sh')
#         print('  ')

In [17]:
# # # 3.2 g2_map_opt_recal8
# recalnum = 8
# path8 = ['50-0.15','50-0.25','50-0.5','200-0.1','400-0.1','400-0.4','400-0.5','600-0.2','800-0.2',
#         '1000-0.1','1000-0.15','1000-0.2','1000-0.25','1000-0.3','1000-0.35','1000-0.4','1000-0.45',
#          '1000-0.5']
# for p8 in path8:
#     print('cd /work/mse-minzw/lrz/g1/g2/zata_thetas/{}'.format(p8))
#     print('mkdir recal{}'.format(recalnum))
#     print('cd recal{}'.format(recalnum))
#     print('cp ../pred.py .')
#     print('cp ../config.ini .')
#     print('cp ../fpParas.dat .')
#     print('cp ../run-pred.sh .')
#     print('cp ../run-pyamff.sh .')
#     print('bsub < run-pyamff.sh')
#     print('  ')


# path8 = ['50-0.15','50-0.25','50-0.5','200-0.1','400-0.1','400-0.4','400-0.5','600-0.2','800-0.2',
#         '1000-0.1','1000-0.15','1000-0.2','1000-0.25','1000-0.3','1000-0.35','1000-0.4','1000-0.45',
#          '1000-0.5']
# for p8 in path8:
#     print('cd /work/mse-minzw/lrz/g1/g2/zata_thetas/{}/recal8'.format(p8))


#     print('bsub < run-pred.sh')
#     print('  ')

In [10]:
# 3.2 g2_map_opt_recal9
# recalnum = 9
# num_zeta = [200, 50, 100] # 2个
# num_thetas = [0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5] # 9个

# for zeta in num_zeta:
#     for ts in num_thetas:
#         print('cd /work/mse-minzw/lrz/g1/g2/zata_thetas/{}-{}'.format(zeta, ts))
#         print('mkdir recal{}'.format(recalnum))
#         print('cd recal{}'.format(recalnum))
#         print('cp ../pred.py .')
#         print('cp ../config.ini .')
#         print('cp ../fpParas.dat .')
#         print('cp ../run-pred.sh .')
#         print('cp ../run-pyamff.sh .')
#         print('bsub < run-pyamff.sh')
#         print('  ')


# recalnum = 9
# num_zeta = [200, 50, 100] # 2个
# num_thetas = [0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5] # 9个

# for zeta in num_zeta:
#     for ts in num_thetas:
#         print('cd /work/mse-minzw/lrz/g1/g2/zata_thetas/{}-{}/recal9'.format(zeta, ts))
#         print('bsub < run-pred.sh')
#         print('  ')

In [23]:
# 3.2 g2_map_opt_recal10
recalnum = 10
num_zeta = [600,800,1000] #2个
num_thetas = [0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5] # 9个

# for zeta in num_zeta:
#     for ts in num_thetas:
#         print('cd /work/mse-minzw/lrz/g1/g2/zata_thetas/{}-{}'.format(zeta, ts))
#         print('mkdir recal{}'.format(recalnum))
#         print('cd recal{}'.format(recalnum))
#         print('cp ../pred.py .')
#         print('cp ../config.ini .')
#         print('cp ../fpParas.dat .')
#         print('cp ../run-pred.sh .')
#         print('cp ../run-pyamff.sh .')
#         print('bsub < run-pyamff.sh')
#         print('  ')

# for zeta in num_zeta:
#     for ts in num_thetas:
#         print('cd /work/mse-minzw/lrz/g1/g2/zata_thetas/{}-{}/recal10'.format(zeta, ts))
#         print('bsub < run-pred.sh')
#         print('  ')

In [32]:
# #3 补充zata = 0.1 and 1  
# # 发现zata小于1的，算不了。现在最小为1.
# num_zeta = [1] 
# num_thetas = [0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5]
# for zeta in num_zeta:
#     for ts in num_thetas:
#         print('cd /work/mse-minzw/lrz/g1/g2/zata_thetas/{}-{}'.format(zeta, ts))
# #         print('cp /work/mse-minzw/lrz/g1/g2/zata_thetas/50-{}/config.ini .'.format(ts))
# #         print('cp /work/mse-minzw/lrz/g1/g2/zata_thetas/50-{}/fpParas.dat .'.format(ts))
# #         print('cp /work/mse-minzw/lrz/g1/g2/zata_thetas/50-{}/run-pyamff.sh .'.format(ts))
# #         print('cp /work/mse-minzw/lrz/g1/g2/zata_thetas/50-{}/pred.py .'.format(ts))
# #         print('cp /work/mse-minzw/lrz/g1/g2/zata_thetas/50-{}/run-pred.sh .'.format(ts))
# #         print('')
#         print('bsub < run-pred.sh')



In [49]:
# #3 补充zata = 0.1 and 1  
# # 发现zata小于1的，算不了。现在最小为1.
# num_zeta = [1] 
# num_thetas = [0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5]
# for zeta in num_zeta:
#     for ts in num_thetas:
#         print('cd /work/mse-minzw/lrz/g1/g2/zata_thetas/1-{}/recal1'.format(ts))
#         print('bsub < run-pred.sh')
#         print('  ')

In [3]:
#3 d_thetas = 0.55 and 0.6
# 补充d_thetas = 0.55 and 0.6 ，寻找边界
# num_zeta = [1, 10, 50, 100, 200, 400, 600, 800, 1000] 
# num_thetas = [0.55, 0.6]
# for zeta in num_zeta:
#     for ts in num_thetas:
#         print('cd /work/mse-minzw/lrz/g1/g2/zata_thetas/{}-{}'.format(zeta, ts))
#         print('cp /work/mse-minzw/lrz/g1/g2/zata_thetas/data_all/fpParas_all/{}-{}.dat .'.format(zeta, ts))
#         print('mv {}-{}.dat fpParas.dat'.format(zeta, ts))
#         print('cp /work/mse-minzw/lrz/g1/g2/zata_thetas/400-0.1/config.ini .')
#         print('cp /work/mse-minzw/lrz/g1/g2/zata_thetas/400-0.1/run-pyamff.sh .')
#         print('cp /work/mse-minzw/lrz/g1/g2/zata_thetas/400-0.1/pred.py .')
#         print('cp /work/mse-minzw/lrz/g1/g2/zata_thetas/400-0.1/run-pred.sh .')
#         print('bsub < run-pyamff.sh')
#         print('  ')

# for zeta in num_zeta:
#     for ts in num_thetas:
#         print('cd /work/mse-minzw/lrz/g1/g2/zata_thetas/{}-{}/recal1'.format(zeta, ts))
#         print('bsub < run-pred.sh')
#         print('  ')

In [ ]:
#####继续优化！！！！！！3月27日星期三

In [2]:
# 3.1 g2_map_opt_recal327_1
# num_ = 3271
# data_p_f = pd.read_excel('/Users/renzhe/Desktop/g2/g2_map_opt.xlsx',sheet_name='g2_opt')
# for i in range(1,len(data_p_f)-1):
#     _a = data_p_f.iloc[i-1,-1]
#     a  = data_p_f.iloc[i+0,-1]
#     a_ = data_p_f.iloc[i+1,-1]
#     if (_a>a) and (a_>a):
#         _pathi1 = data_p_f.iloc[i-1,0]
#         _pathi2 = data_p_f.iloc[i-1,1]
#         print('cd /work/mse-minzw/lrz/g1/g2/zata_thetas/{}-{}'.format(_pathi1, _pathi2))
#         print('mkdir recal{}'.format(num_))
#         print('cd recal{}'.format(num_))
#         print('cp ../pred.py .')
#         print('cp ../config.ini .')
#         print('cp ../fpParas.dat .')
#         print('cp ../run-pred.sh .')
#         print('cp ../run-pyamff.sh .')
#         print('bsub < run-pyamff.sh')
#         print('  ')
        
#         pathi1_ = data_p_f.iloc[i+1,0]
#         pathi2_ = data_p_f.iloc[i+1,1]
#         print('cd /work/mse-minzw/lrz/g1/g2/zata_thetas/{}-{}'.format(pathi1_, pathi2_))
#         print('mkdir recal{}'.format(num_))
#         print('cd recal{}'.format(num_))
#         print('cp ../pred.py .')
#         print('cp ../config.ini .')
#         print('cp ../fpParas.dat .')
#         print('cp ../run-pred.sh .')
#         print('cp ../run-pyamff.sh .')
#         print('bsub < run-pyamff.sh')
#         print('  ')        
        

# for i in range(1,len(data_p_f)-1):
#     _a = data_p_f.iloc[i-1,-1]
#     a  = data_p_f.iloc[i+0,-1]
#     a_ = data_p_f.iloc[i+1,-1]
#     if (_a>a) and (a_>a):
#         _pathi1 = data_p_f.iloc[i-1,0]
#         _pathi2 = data_p_f.iloc[i-1,1]
#         print('cd /work/mse-minzw/lrz/g1/g2/zata_thetas/{}-{}/recal{}'.format(_pathi1, _pathi2, num_))
#         print('bsub < run-pred.sh')
#         print('    ')        
#         pathi1_ = data_p_f.iloc[i+1,0]
#         pathi2_ = data_p_f.iloc[i+1,1]
#         print('cd /work/mse-minzw/lrz/g1/g2/zata_thetas/{}-{}/recal{}'.format(pathi1_, pathi2_, num_))
#         print('bsub < run-pred.sh')
#         print('    ')    

In [73]:
# # 3.1 g2_map_opt_recal328_1
# num_ = 3281
# data_p_f = pd.read_excel('/Users/renzhe/Desktop/g2/g2_map_opt.xlsx',sheet_name='g2_opt')
# for i in range(1,len(data_p_f)-1):
#     _a = data_p_f.iloc[i-1,-1]
#     a  = data_p_f.iloc[i+0,-1]
#     a_ = data_p_f.iloc[i+1,-1]
#     if (_a<a) and (a_<a):
#         _pathi1 = data_p_f.iloc[i,0]
#         _pathi2 = data_p_f.iloc[i,1]
#         print('cd /work/mse-minzw/lrz/g1/g2/zata_thetas/{}-{}'.format(_pathi1, _pathi2))
#         print('mkdir recal{}'.format(num_))
#         print('cd recal{}'.format(num_))
#         print('cp ../pred.py .')
#         print('cp ../config.ini .')
#         print('cp ../fpParas.dat .')
#         print('cp ../run-pred.sh .')
#         print('cp ../run-pyamff.sh .')
#         print('bsub < run-pyamff.sh')
#         print('  ')
       
        
# # 3.1 g2_map_opt_recal328_1
# num_ = 3281
# data_p_f = pd.read_excel('/Users/renzhe/Desktop/g2/g2_map_opt.xlsx',sheet_name='g2_opt')
# for i in range(1,len(data_p_f)-1):
#     _a = data_p_f.iloc[i-1,-1]
#     a  = data_p_f.iloc[i+0,-1]
#     a_ = data_p_f.iloc[i+1,-1]
#     if (_a<a) and (a_<a):
#         _pathi1 = data_p_f.iloc[i,0]
#         _pathi2 = data_p_f.iloc[i,1]
#         print('cd /work/mse-minzw/lrz/g1/g2/zata_thetas/{}-{}/recal{}'.format(_pathi1, _pathi2, num_))
#         print('bsub < run-pred.sh')
#         print('  ')

cd /work/mse-minzw/lrz/g1/g2/zata_thetas/1-0.3/recal3281
bsub < run-pred.sh
  
cd /work/mse-minzw/lrz/g1/g2/zata_thetas/1-0.45/recal3281
bsub < run-pred.sh
  
cd /work/mse-minzw/lrz/g1/g2/zata_thetas/1-0.55/recal3281
bsub < run-pred.sh
  
cd /work/mse-minzw/lrz/g1/g2/zata_thetas/10-0.25/recal3281
bsub < run-pred.sh
  
cd /work/mse-minzw/lrz/g1/g2/zata_thetas/10-0.55/recal3281
bsub < run-pred.sh
  
cd /work/mse-minzw/lrz/g1/g2/zata_thetas/50-0.15/recal3281
bsub < run-pred.sh
  
cd /work/mse-minzw/lrz/g1/g2/zata_thetas/50-0.25/recal3281
bsub < run-pred.sh
  
cd /work/mse-minzw/lrz/g1/g2/zata_thetas/50-0.6/recal3281
bsub < run-pred.sh
  
cd /work/mse-minzw/lrz/g1/g2/zata_thetas/100-0.25/recal3281
bsub < run-pred.sh
  
cd /work/mse-minzw/lrz/g1/g2/zata_thetas/100-0.45/recal3281
bsub < run-pred.sh
  
cd /work/mse-minzw/lrz/g1/g2/zata_thetas/100-0.6/recal3281
bsub < run-pred.sh
  
cd /work/mse-minzw/lrz/g1/g2/zata_thetas/200-0.4/recal3281
bsub < run-pred.sh
  
cd /work/mse-minzw/lrz/g1/g2/za

In [11]:
# 3.1 g2_map_opt_recal328_1
num_ = 3291
lll = ['1000-0.5','1000-0.6','1000-0.3','1000-0.15',]
# for ll in lll:
#     print('cd /work/mse-minzw/lrz/g1/g2/zata_thetas/{}'.format(ll))
#     print('mkdir recal{}'.format(num_))
#     print('cd recal{}'.format(num_))
#     print('cp ../pred.py .')
#     print('cp ../config.ini .')
#     print('cp ../fpParas.dat .')
#     print('cp ../run-pred.sh .')
#     print('cp ../run-pyamff.sh .')
#     print('bsub < run-pyamff.sh')
#     print('  ')        
# for ll in lll:
#     print('cd /work/mse-minzw/lrz/g1/g2/zata_thetas/{}/recal{}'.format(ll,num_))
#     print('bsub < run-pred.sh')
#     print('  ')

In [1]:
import numpy as np

# 定义一个二次函数作为真实数据生成器
def quadratic_function(x1, x2):
    return np.random.rand(len(x1))



# 生成随机二维输入数据
X = np.random.uniform(x1_lim[0], x1_lim[1], size=(n_samples, 2))
X[:, 1] = np.random.uniform(x2_lim[0], x2_lim[1], size=n_samples)

# 计算对应的输出值
y = quadratic_function(X[:, 0], X[:, 1])


In [2]:
# 设置数据生成参数
n_samples = 20
x1_lim = (-100, 100)
x2_lim = (-100, 100)

In [8]:
X = np.random.uniform(x1_lim[0], x1_lim[1], size=(n_samples, 2))
X[:, 1] = np.random.uniform(x2_lim[0], x2_lim[1], size=n_samples)

In [9]:
X

array([[ 66.22707809, -25.55484047],
       [-99.31158153, -12.63469363],
       [ 76.33382146,  93.45638618],
       [-71.7159306 ,   6.8142728 ],
       [-17.86308731, -26.85764732],
       [-82.02706844, -81.0555551 ],
       [-12.62771469,  78.40806037],
       [-58.44230187,  38.05800582],
       [-95.38038282,  66.50962607],
       [-40.77295081,   3.60392041],
       [-22.25132798, -36.11648511],
       [ -0.41019189, -34.85843153],
       [ 62.12752772,  15.37353992],
       [ 18.64644977, -60.41458438],
       [ 97.79625374, -52.01624768],
       [ 67.63830803, -97.51767575],
       [ 69.31862562,  78.02644265],
       [-58.53029925,  48.50445418],
       [-51.16234968, -97.10348613],
       [-95.58274962,  -2.23394229]])

In [10]:
X[:, 1]

array([-25.55484047, -12.63469363,  93.45638618,   6.8142728 ,
       -26.85764732, -81.0555551 ,  78.40806037,  38.05800582,
        66.50962607,   3.60392041, -36.11648511, -34.85843153,
        15.37353992, -60.41458438, -52.01624768, -97.51767575,
        78.02644265,  48.50445418, -97.10348613,  -2.23394229])

In [11]:
y = quadratic_function(X[:, 0], X[:, 1])

In [12]:
y

array([0.61639326, 0.89071262, 0.41174719, 0.69390862, 0.77248302,
       0.33604458, 0.26413545, 0.58814544, 0.49641881, 0.33768792,
       0.2916864 , 0.14957144, 0.2623006 , 0.91406967, 0.4899272 ,
       0.34883644, 0.47507905, 0.05328848, 0.33133639, 0.839284  ])